### Run the code below only if you are using your own RL model

In [ ]:
import numpy as np
import os, json
from argparse import Namespace
from flat_reach_RL import *
from nnet_util import torch_model_to_nnet, onnx_to_nnet
import onnxruntime

In [ ]:
# TODO change
demo_path = "/Users/changliuliu/Documents/GitHub/Composable_Agent_Toolbox/examples/output/FlatReach_near_C200_H32_Nm100.0_lr0.001"
demo_ep = 300

with open(os.path.join(demo_path, "args.json"), "r") as infile:
    config = json.load(infile)
args_saved = Namespace(**config)

args_saved.demo = True
args_saved.demo_path = demo_path
args_saved.demo_ep = demo_ep

args = args_saved

STATE_DIMENSION = 8
ACTION_SPACE_SIZE = 17

# TODO change
device = 'cpu'
# device = 'cuda'

In [ ]:
def state_to_tensor(s, device):
    '''
        [1, dS]
    '''
    return torch.from_numpy(np.asarray(s)).unsqueeze(0).float().to(device)

In [ ]:
Q = QNetwork(env_name=args.env_name,
            state_dim=STATE_DIMENSION,
            num_of_actions=ACTION_SPACE_SIZE,
            hidden_size=args.hidden_size).to(device)

checkpt_path = os.path.join(args.demo_path, 'checkpts', '{}_{}.pt'.format(args.env_name, args.demo_ep))
Q.load_model_weights(checkpt_path)
Q.eval()

In [ ]:
# TODO change if necessary
ANGLE_DISCRETE_SIZE = 8
ACC_MIN = 50
ACC_MAX = 100
ACC_DISCRETE_SIZE = 2
ACTION_SPACE_SIZE = 1 + ANGLE_DISCRETE_SIZE * ACC_DISCRETE_SIZE

def get_action(Q, state, device):

    action_id = Q(state_to_tensor(state, device)).argmax().item()

    if action_id == 0:
        action = [0, 0]
    else:
        action_id_non_zero = action_id - 1
        angle = 2*np.pi / ANGLE_DISCRETE_SIZE * (action_id_non_zero % ANGLE_DISCRETE_SIZE)
        mag = np.floor(action_id_non_zero/ANGLE_DISCRETE_SIZE)*(ACC_MAX-ACC_MIN)/(ACC_DISCRETE_SIZE-1) + ACC_MIN
        action = [mag*np.cos(angle), mag*np.sin(angle)]
    
    return action_id, np.asarray(action).reshape(-1)

# numerical examples
# state = [goal_rel_pos, goal_rel_vel, obs_rel_pos, obs_rel_vel]

# When obstacle is far, changing goal distance should not change action
state = [10, 0, -1, 0, 0, 50, 0, 0]
print(get_action(Q, state, device))

state = [5, 0, -1, 0, 0, 50, 0, 0]
print(get_action(Q, state, device))

# When obstacle is not on the way to goal, changing obstacle vel should not change action
state = [10, 0, -1, 0, 0, 50, 0, 0]
print(get_action(Q, state, device))

state = [10, 0, -1, 0, 0, 50, 0, -1]
print(get_action(Q, state, device))

# When obstacle is on the way to goal, changing goal distance should not change action
state = [20, 0, -1, 0, 10, 0, -2, 0]
print(get_action(Q, state, device))

state = [25, 0, -1, 0, 10, 0, -2, 0]
print(get_action(Q, state, device))


In [ ]:
torch_model_to_nnet(Q, f"{demo_ep}.nnet", decimal=8) # saving 8 digits. More can be used, but that requires more more space.

### Homework 4 Main Code

In [ ]:
# Test nnet in julia using the following code:
# the output could be slightly different due to digits used for saving nnet

using NeuralVerification
# TODO change if necessary
net = read_nnet("300.nnet")

In [ ]:
function get_result(solver, problem)  
    model = NeuralVerification.Model(solver)
    z = NeuralVerification.init_vars(model, problem.network, :z, with_input=true)
    δ = NeuralVerification.init_vars(model, problem.network, :δ, binary=true)
    # get the pre-activation bounds:
    model[:bounds] = NeuralVerification.get_bounds(problem, before_act=true)
    model[:before_act] = true
    NeuralVerification.add_set_constraint!(model, problem.input, first(z))
    NeuralVerification.add_complementary_set_constraint!(model, problem.output, last(z))
    NeuralVerification.encode_network!(model, problem.network, NeuralVerification.BoundedMixedIntegerLP())
    o = NeuralVerification.max_disturbance!(model, first(z) - problem.input.center)
    NeuralVerification.optimize!(model)
    if NeuralVerification.termination_status(model) == NeuralVerification.OPTIMAL
        return (1,NeuralVerification.value(first(z)))
    else
        return (0,0.0)
    end
end

In [ ]:
# Case 1: when the obstacle is far, changing goal distance should not change action
input = [10.0, 0.0, -1.0, 0.0, 0.0, 50.0, 0.0, 0.0]

# Check the desired action
output = NeuralVerification.compute_output(net, input)
desired_action = argmax(output) # Note Julia's index starts from 1

# Now check whether the input can tolerate some perturbation
radius = [10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] # This parameter is tunable
input_set  = NeuralVerification.Hyperrectangle(input, radius)

# Choose solver
solver = MIPVerify()

# ---------- Problem 1 ----------
# Now we check if the desired_action remains the same for all inputs in the input_set
# TODO check all non-optimal actions according to the model
# Hint: the following code block checks action i=0 against desired_action, assuming action 0 != desired_action
i = 0
vec = zeros(1,17)
vec[i] = 1.0
vec[desired_action] = -1.0
output_set = NeuralVerification.HPolytope(vec, [0.0])
problem = Problem(net, input_set, output_set)
result = solve(solver, problem)
println("action pair: ",i, ",", desired_action, ": ",result.status)

# ---------- Problem 2 ----------
# Now let us check at which distance the obstacle can affect the action (make the adjacent action optimal)
# TODO initialize variables (e.g., bounds for binary search)
# Hint: we are searching for an obstacle y-coordinate in [0, 50]. The x-coordinate is always 0.

# TODO binary search loop
# while CONDITION
    # TODO set obstacle distance
    
    # TODO define input_set, get model output
    
    # TODO check non-optimal actions against model output
    # Note: use get_result() instead of solve() for this question
    # Syntax
    #       status, x = get_result(solver, problem)
    #       status: 0 (verification success) / 1 (verification failed)
    #       x: counter example, if status is 1
    
    # TODO conditions for updating binary search bounds
    
# end

# TODO Now we verify that if the obstacle's distance is above dmax (but within 50), then the change of goal position won't affect the action
# Hint: this should be very similar to problem 1, but what should be the input and radius?


# ---------- Problem 3 ----------
# We now choose different solvers
input = [10.0, 0.0, -1.0, 0.0, 0.0, 50.0, 0.0, 0.0]
radius = [10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] # This parameter is tunable
input_set  = NeuralVerification.Hyperrectangle(input, radius)

# TODO tune
solver = ReluVal(max_iter = 50) # max_iter is a tunable parameter. there will be fewer unknowns if we set max_iter to be big enough

# TODO verify case 1 using ReluVal solver, similar to problem 1


In [ ]:
# Case 2: When obstacle is not on the way to goal, changing obstacle vel should not change action
input = [10, 0, -1, 0, 5, 50, 0, 0]

# Check the desired action
output = NeuralVerification.compute_output(net, input)
desired_action = argmax(output) # Note Julia's index starts from 1

# Now check whether the input can tolerate some perturbation
radius = [0, 0, 0, 0, 0, 0, 0, 10] # This parameter is tunable
input_set  = NeuralVerification.Hyperrectangle(input, radius)

# Choose solver
solver = MIPVerify()

# ---------- Problem 1 ----------
# Now we check if the desired_action remains the same for all inputs in the input_set

# ---------- Problem 2 ----------
# Now let us check at which distance the obstacle can affect the action (make the adjacent action optimal)
# Now we verify that if the obstacle's distance is above dmax, then the change of obstacle velocity won't affect the action

# ---------- Problem 3 ----------
# Choose different solvers

In [ ]:
# Case 3: When obstacle is on the way to goal, changing goal distance should not change action
input = [20, 0, -1, 0, 10, 0, -2, 0]

# Check the desired action
output = NeuralVerification.compute_output(net, input)
desired_action = argmax(output) # Note Julia's index starts from 1

# Now check whether the input can tolerate some perturbation
radius = [10, 0, 0, 0, 0, 0, 0, 0] # This parameter is tunable
input_set  = NeuralVerification.Hyperrectangle(input, radius)

# Choose solver
solver = MIPVerify()

# ---------- Problem 1 ----------
# Now we check if the desired_action remains the same for all inputs in the input_set

# ---------- Problem 2 ----------
# Now let us check at which distance the obstacle can affect the action (make the adjacent action optimal)
# Now we verify that if the obstacle's distance is below dmin, then the change of goal distance won't affect the action

# ---------- Problem 3 ----------
# Choose different solvers